# Estymacja cen opcji europejskich i barierowych metodą Monte Carlo
## Wprowadzenie do symulacji i metod Monte Carlo
### Mikołaj Płóciniczak, 2026

In [10]:
import pickle
import pandas as pd
from pathlib import Path

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

PLOTS_DIR = Path("..").resolve().parent / "plots"
ROOT = Path.cwd().resolve()
while not (ROOT / "parameters.py").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

def show_code(path):
    code = (ROOT / path).read_text()
    formatter = HtmlFormatter(style="native")  # works well in dark mode
    styles = formatter.get_style_defs('.highlight')
    html = f"<style>{styles}</style>" + highlight(code, PythonLexer(), formatter)
    return HTML(html)


# 1. Wprowadzenie

Celem tego raportu jest estymacja cen opcji europejskiej oraz dyskretnej opcji barierowej typu up-and-out metodą Monte Carlo. Rozważamy proces ceny akcji opisany geometrycznym ruchem Browna (GBM) o postaci
$$
S(t) = S(0)\exp(\mu^* t + \sigma B(t)),
$$
gdzie $B(t)$ jest ruchem Browna, a parametry modelu są ustalone: $r = 0{,}05$, $\sigma = 0{,}125$, $\mu^* = r - \sigma^2/2 = -0{,}0125$, $S(0) = 100$ oraz $K = 100$.

W projekcie porównujemy kilka estymatorów Monte Carlo:
- klasyczny estymator typu Crude Monte Carlo,
- estymator ze stratyfikacją (dla podziału proporcjonalnego i optymalnego),
- estymator z antytetycznymi zmiennymi (dla $n = 1$),
- estymator z wykorzystaniem zmiennej kontrolnej (dla $n = 1$).

Dla przypadku opcji europejskiej ($C = \infty$) wyniki symulacji porównujemy z wartością dokładną daną wzorem Blacka–Scholesa. Dla skończonych wartości bariery $C$ analizujemy wpływ bariery na wartość opcji oraz efektywność poszczególnych technik redukcji wariancji.

# 2. Metody symulacyjne

W tej części opisujemy sposób generowania ścieżek procesu GBM oraz konstrukcję różnych estymatorów Monte Carlo użytych w projekcie.

Poniżej przedstawiono krótkie omówienie modeli oraz metod, na których opiera się dalsza analiza.

## 2.1. Model geometrycznego ruchu Browna

Model geometrycznego ruchu Browna (GBM) opisuje ewolucję ceny aktywa jako
$$
S(t) = S(0)\exp(\mu^* t + \sigma B(t)),
$$
gdzie ruch Browna $B(t)$ stanowi źródło losowości. W projekcie ścieżki GBM powstają na podstawie wygenerowanego wektora wartości ruchu Browna o odpowiedniej kowariancji.  
Dla metod redukcji wariancji, takich jak stratyfikacja, stosujemy zmodyfikowane procedury generowania BM.

In [6]:
show_code("models/simulate_gbm_paths.py")

## 2.2. Definicja wypłaty opcji

W projekcie rozważamy dwie wypłaty: europejską oraz dyskretną barierową typu up-and-out.
Dla bariery $C < \infty$ wypłata wynosi zero, jeśli ścieżka ceny choć raz przekroczy poziom $C$; w przeciwnym razie jest to zdyskontowana wartość $(S(1) - K)_+$.  
Gdy $C = \infty$, otrzymujemy zwykłą opcję europejską.

## 2.3. Estymator Crude Monte Carlo

Podstawowy estymator Monte Carlo polega na generowaniu wielu niezależnych ścieżek GBM, obliczeniu wypłaty dla każdej z nich, a następnie uśrednieniu wyników. Jest to metoda referencyjna, do której porównujemy bardziej zaawansowane techniki redukcji wariancji.

## 2.4. Estymator stratyfikowany

W tym projekcie estymator stratyfikowany polega na estymacji wartości $I_{n,C}$ poprzez niezależne symulowanie ścieżek GBM warunkowo na przynależność odpowiadającego im wektora ruchu Browna do zadanych stratum. Wynik końcowy otrzymywany jest jako ważona średnia estymatorów cząstkowych z poszczególnych stratum, co pozwala na kontrolę rozkładu symulowanych trajektorii i redukcję wariancji względem crude Monte Carlo.

W dalszej części pracy rozważane będą dwie wersje estymatora stratyfikowanego, różniące się sposobem alokacji liczby symulacji pomiędzy poszczególne straty.

### 2.4.1. Rozmiar próby proporcjonalny

W przypadku podziału próby proporcjonalnego liczba symulacji w każdej stracie jest proporcjonalna do jej prawdopodobieństwa. Jest to naturalne rozszerzenie estymatora crude Monte Carlo, które zapewnia nieobciążoność estymatora, jednak nie wykorzystuje informacji o zróżnicowaniu wariancji wypłaty pomiędzy poszczególnymi stratum.

### 2.4.2 Rozmiar próby optymalny

## 2.5. Estymator antytetyczny

Zgodnie ze slajdami z wykładu estymujemy
$I=\mathbb{E}[Y]$, gdzie w naszym przypadku
$$
Y = A(Z)=e^{-r}\big(S(0)e^{\mu^*+\sigma Z}-K\big)_+,\qquad Z\sim\mathcal N(0,1).
$$

Standardowy estymator Monte Carlo ma wariancję
$$
\mathrm{Var}(\hat I_R^{\mathrm{CMC}})=\frac{\mathrm{Var}(A(Z))}{R}.
$$

Metoda antytetyczna polega na generowaniu par $(Z,-Z)$, co odpowiada konstrukcji
$Z=\Phi^{-1}(U)$ oraz $-Z=\Phi^{-1}(1-U)$ dla $U\sim U(0,1)$. Estymator ma postać
$$
\hat I_R^{\mathrm{anti}}
=\frac{1}{R}\sum_{i=1}^{R/2}\big(A(Z_i)+A(-Z_i)\big).
$$

Wariancja estymatora antytetycznego spełnia zależność
$$
\mathrm{Var}(\hat I_R^{\mathrm{anti}})
=
\mathrm{Var}(\hat I_R^{\mathrm{CMC}})
\big(1+\mathrm{Corr}(A(Z),A(-Z))\big).
$$

Funkcja $A(Z)$ jest niemalejąca względem $Z$, ponieważ jest złożeniem funkcji
wykładniczej i funkcji $(\cdot)_+$. Zgodnie z lematami ze slajdów, dla funkcji
monotonicznych zachodzi
$$
\mathrm{Corr}(A(Z),A(-Z))\le 0.
$$

W konsekwencji
$$
\mathrm{Var}(\hat I_R^{\mathrm{anti}})
\le
\mathrm{Var}(\hat I_R^{\mathrm{CMC}}),
$$
co uzasadnia, że zmniejszenie wariancji zmiennych wejściowych $(Z,-Z)$
przenosi się na zmniejszenie wariancji funkcji wypłaty $A(Z)$.

## 2.6. Estymator zmiennej kontrolnej

W metodzie *control variate* wykorzystujemy dodatkową zmienną losową $X$, dla której znana jest wartość oczekiwana oraz która jest silnie skorelowana z estymowaną zmienną $Y = A_{1,\infty}$.  
W niniejszym projekcie jako zmienną kontrolną przyjmujemy
$$
X = B(1),
$$
czyli wartość ruchu Browna w chwili $t=1$. Z własności ruchu Browna wynika, że $B(1)\sim N(0,1), \qquad \mathbb{E}[X]=0.$ Wybór ten jest naturalny, ponieważ zarówno cena $S(1)$, jak i payoff opcji są funkcjami $B(1)$, co implikuje istotną korelację pomiędzy $X$ i $Y$.

#### Konstrukcja estymatora *control variate*

Dla $R$ niezależnych symulacji generujemy próbki
$$
X_j = B(1)_j \sim N(0,1), \qquad
Y_j = e^{-r}\big(S(0)e^{\mu^* + \sigma X_j} - K\big)_+,
\quad j=1,\dots,R.
$$

Estymator z zastosowaniem zmiennej kontrolnej definiujemy jako
$$
\hat I^{CV}_R
=
\hat I^{CMC}_R
+
\hat c \left(\frac{1}{R}\sum_{j=1}^R X_j - \mathbb{E}[X]\right).
$$
Ponieważ $\mathbb{E}[X]=0$, wzór upraszcza się do
$$
\hat I^{CV}_R
=
\frac{1}{R}\sum_{j=1}^R Y_j
+
\hat c\,\frac{1}{R}\sum_{j=1}^R X_j.
$$

Współczynnik $\hat c$ estymujemy z próby zgodnie z klasyczną formułą
$$
\hat c
=
-
\frac{\widehat{\mathrm{Cov}}(Y,X)}{\widehat{\mathrm{Var}}(X)},
$$
gdzie
$$
\widehat{\mathrm{Cov}}(Y,X)
=
\frac{1}{R-1}\sum_{j=1}^R (Y_j-\bar Y)(X_j-\bar X),
\qquad
\widehat{\mathrm{Var}}(X)
=
\frac{1}{R-1}\sum_{j=1}^R (X_j-\bar X)^2.
$$

# 3. Black-Scholes Benchmark

W przypadku $n = 1$ i $C = \infty$ rozważany instrument redukuje się do europejskiej opcji call, dla której znana jest wartość dokładna. W projekcie służy ona jako punkt odniesienia do porównania jakości estymatorów Monte Carlo z wynikami analitycznymi. Wartość ta dana jest przez formułę Blacka–Scholesa:
$$
I_{1,\infty} = \mathbb{E}\!\left[e^{-r}(S(1)-K)^+\right]
            = S(0)\,\Phi(d_1) - K e^{-r}\,\Phi(d_2),
$$
gdzie
$$
d_1 = \frac{1}{\sigma}\!\left(\log\frac{S(0)}{K} + r + \frac{\sigma^2}{2}\right),
\qquad
d_2 = d_1 - \sigma,
$$
a $\Phi$ oznacza dystrybuantę standardowego rozkładu normalnego.

# 4. Ustawienia eksperymentów

## 4.1. Liczba symulacji R

Dla estymatora  
$$
\hat{I} = \frac{1}{R}\sum_{i=1}^R Y_i
$$  
błąd standardowy wynosi  
$$
\mathrm{SE}(\hat{I}) = \frac{\sigma}{\sqrt{R}},
$$  
czyli dokładność rośnie powoli (proporcjonalnie do $1/\sqrt{R}$). Dobór $R$ musi więc równoważyć precyzję i koszt obliczeń.

W profesjonalnej wycenie opcji Monte Carlo używa się zwykle $10^4\!-\!10^5$ symulacji dla instrumentów o umiarkowanej wariancji wypłaty. Taki rząd wielkości zapewnia stabilność wyników i błędy standardowe na poziomie akceptowalnym w praktyce rynkowej.

W projekcie przyjęliśmy  
$$
R = 100{,}000,
$$  
co mieści się w typowym zakresie stosowanym w finansach i gwarantuje wiarygodne oszacowania przy rozsądnym czasie obliczeń. W metodach redukcji wariancji taka wartość jest nawet nadmiarowa, ponieważ ich estymatory mają mniejszą wariancję.

Warto zauważyć, że wartość $R = 100{,}000$, dobrana pod kątem opcji europejskiej (czyli bez bariery), jest tym bardziej wystarczająca w przypadku opcji z barierą. Jeśli oznaczymy wypłatę opcji europejskiej jako $A_\infty$ oraz wypłatę opcji z barierą $C$ jako $A_C$, to zachodzi  
$$
A_C = A_\infty \cdot \mathbf{1}_{\{\text{bariera nie została trafiona}\}},
$$  
czyli $A_C \le A_\infty$ dla każdej realizacji. Ponadto  
$$
A_C^2 \le A_\infty^2 \quad \text{oraz} \quad \mathbb{E}[A_C^2] < \mathbb{E}[A_\infty^2],
$$  
więc wariancja opcji barierowej spełnia  
$$
\mathrm{Var}(A_C) = \mathbb{E}[A_C^2] - \mathbb{E}[A_C]^2 < \mathbb{E}[A_\infty^2] - \mathbb{E}[A_\infty]^2 = \mathrm{Var}(A_\infty).
$$  
Zatem wariancja wypłat (czyli $\sigma^2$ używana w oszacowaniu błędu standardowego) jest mniejsza w przypadku opcji z barierą, co oznacza, że błąd standardowy $\mathrm{SE}(\hat{I})$ również jest mniejszy niż dla opcji europejskiej — przy tej samej liczbie symulacji $R$. Przyjęta wartość $R = 100{,}000$ jest więc tym bardziej wystarczająca dla przypadku z barierą.


## 4.2 Dyskretyzacja osi czasu (n)

Symulacje GBM obserwujemy w punktach
$$
t_k = \frac{k}{n},
$$
więc $n$ jest parametrem dyskretyzacji wpływającym na wykrywanie przekroczeń bariery.
Większe $n$ oznacza dokładniejszą obserwację ścieżki, ale też wyższy koszt obliczeń
(złożoność $\sim R \cdot n$).

W praktycznej wycenie opcji barierowych **nie stosuje się bariery ciągłej** — kontrakt
określa konkretne daty monitorowania, zwykle dzienne (ok. 250/rok), tygodniowe (ok. 52/rok)
lub miesięczne (ok. 12/rok). Profesjonalne modele odzwierciedlają właśnie taką dyskretną
strukturę, a nie idealizację ciągłą.

Traktując $T = 1$ jako rok, przyjęcie
$$
n = 12
$$
jest zgodne z **typową częstotliwością miesięczną** i odpowiada praktycznym standardom
wyceny dyskretnych barier, zapewniając jednocześnie rozsądny koszt obliczeń.

## 4.3. Wartości barier $C_1$ i $C_2$

Wybór dwóch wartości bariery up-and-out — $H = 105$ oraz $H = 120$ — pozwala analizować dwa odmienne reżimy zachowania opcji barierowej przy założeniu $S_0 = 100$. Poziom $H = 105$ reprezentuje barierę bardzo bliską cenie początkowej, co skutkuje wysokim prawdopodobieństwem knock-outu i silną zależnością wyniku od maksymalnych wartości ścieżki. Taka konfiguracja jest szczególnie interesująca numerycznie, ponieważ ujawnia problemy związane z dyskretnym monitoringiem bariery oraz konieczność stosowania korekt, takich jak Brownian Bridge. Jednocześnie stanowi klasyczny przykład „silnie barierowy”, często wykorzystywany w literaturze do testowania jakości metod Monte Carlo.

Z kolei bariera $H = 120$ znajduje się znacznie dalej, co prowadzi do znacznie niższego prawdopodobieństwa aktywacji bariery i sprawia, że opcja zachowuje się bardziej jak instrument vanilla. Dzięki temu przypadek ten jest stabilniejszy numerycznie i pozwala badać, jak zmienia się wycena oraz charakterystyki opcji, gdy bariera ma marginalny wpływ na dynamikę wypłaty. Wspólne rozważenie obu poziomów umożliwia uchwycenie przejścia od silnego do słabego efektu bariery, dając pełniejszy obraz zjawiska i zgodnie z praktyką literaturową tworzy reprezentatywny zestaw przykładów do analiz teoretycznych i numerycznych.

## 4.4. Liczba strat $m$

W metodzie stratyfikacji ważne jest, aby w każdej stracie znajdowała się wystarczająco duża liczba próbek. Jeżeli całkowita liczba symulacji to $R$, to liczba próbek przypadających na jedną stratę wynosi

$$
R_i = \frac{R}{m}.
$$

Gdy $m$ jest zbyt duże, wtedy $R_i$ staje się małe, a średnie warunkowe $\bar{Y}_i$ obliczane w poszczególnych stratach mają wysoką wariancję. W skrajnym przypadku, kiedy $m$ byłoby zbliżone do $R$, każda strata miałaby około jednej próbki, przez co stratyfikacja przestaje działać i sprowadza się do zwykłego Monte Carlo.

W celu zbadania wpływu liczby stratum $m$ na jakość estymacji w metodzie stratyfikacji
rozważamy trzy wartości:
$$
m \in \{10, 20, 50\}.
$$
Dla każdej z nich analizujemy dwa schematy alokacji próby: **alokację proporcjonalną**
oraz **alokację optymalną**. Całkowita liczba symulacji $R$ jest stała, a dla każdej
kombinacji liczby stratum i schematu alokacji wykonujemy **100 niezależnych powtórzeń
estymatora**. Dla każdego powtórzenia obliczamy błąd estymacji jako różnicę pomiędzy
wartością uzyskaną metodą Monte Carlo a dokładną wartością wyznaczoną ze wzoru
Blacka--Scholesa.

In [5]:
hist_path = PLOTS_DIR / "strata_errors_hist.pkl"
box_path = PLOTS_DIR / "strata_errors_box.pkl"

with hist_path.open("rb") as f:
    hist_fig = pickle.load(f)
hist_fig

In [6]:
with box_path.open("rb") as f:
    box_fig = pickle.load(f)
box_fig

In [7]:
opt_hist_path = PLOTS_DIR / "strata_errors_hist_optimal.pkl"
opt_box_path = PLOTS_DIR / "strata_errors_box_optimal.pkl"

with opt_hist_path.open("rb") as f:
    opt_hist_fig = pickle.load(f)
opt_hist_fig

In [8]:
with opt_box_path.open("rb") as f:
    opt_box_fig = pickle.load(f)
opt_box_fig


Na wykresach przedstawiono rozkłady błędów estymacji w postaci histogramów oraz
wykresów pudełkowych. Pierwsze dwa wykresy odpowiadają estymatorom z
**alokacją proporcjonalną**, natomiast kolejne dwa --- estymatorom z
**alokacją optymalną**.

W przypadku alokacji proporcjonalnej przejście z $m=10$ do $m=20$ prowadzi do
widocznego zmniejszenia rozrzutu błędów estymacji. Dalsze zwiększenie liczby stratum
do $m=50$ nie powoduje jednak wyraźnej dodatkowej poprawy --- rozkłady błędów dla
$m=20$ i $m=50$ są zbliżone zarówno pod względem szerokości, jak i położenia
względem zera.

Dla alokacji optymalnej obserwujemy podobny trend. Wartość $m=20$ prowadzi do
wyraźnie skoncentrowanego rozkładu błędów wokół zera. W przypadku $m=50$ rozrzut
błędów pozostaje porównywalny, a w szczególności nie obserwujemy pogorszenia jakości
estymacji. Co więcej, w porównaniu z $m=20$ widoczny jest brak najbardziej skrajnych
obserwacji odstających, co może sugerować nieco większą stabilność rozkładu błędów,
choć różnice te są niewielkie.

Na podstawie przeprowadzonych symulacji można stwierdzić, że zwiększanie liczby
stratum powyżej $m=20$ nie przynosi istotnych korzyści w kontekście redukcji błędu
estymacji, zarówno dla alokacji proporcjonalnej, jak i optymalnej. Jednocześnie nie ma
jednoznacznych przesłanek, aby uznać wybór $m=50$ za gorszy --- obserwowane różnice
pomiędzy $m=20$ a $m=50$ są raczej drugorzędne.

W związku z tym wybór $m=20$ można traktować jako rozsądny kompromis pomiędzy
szczegółowością podziału na straty a stabilnością estymatora przy danym budżecie
symulacyjnym. Wyniki sugerują jednak, że przy zastosowaniu alokacji optymalnej
również większa liczba stratum może być akceptowalna, choć potencjalne zyski są
ograniczone.

## 4.5. Liczba powtórzeń eksperymentu $M$

liczba replikacji eksperymentu $M=1000$

# 5. Wyniki

W analizie przyjęto ustalone wcześniej parametry modelu i opcji $S_0$, $K$, $r$, $\sigma$ oraz $T$. Wszystkie estymatory porównywane są przy tym samym budżecie symulacji $R = 10^5$ trajektorii.

## 5.1. Opcja europejska ($C=\infty,\,n=1$)

W niniejszej części analizy zajmujemy się estymacją ceny europejskiej opcji kupna. Porównujemy tu wszystkie estymatory, analizując ich dokładność i wariancję. Jako punkt odniesienia wykorzystujemy teoretyczną wartość opcji daną wzorem Blacka–Scholesa, która w tym przypadku jest znana w postaci jawnej. Dzięki temu możliwe jest bezpośrednie porównanie estymatorów oraz ocena ich błędu i efektywności numerycznej.

### 5.1.1. Tabela

Tworzymy zbiorczą tabelę porównującą wszystkie rozważane estymatory ceny europejskiej opcji kupna. Zawiera ona średnią estymatę, odchylenie standardowe, obciążenie, błąd średniokwadratowy oraz empiryczne pokrycie 95% przedziału ufności, co pozwala bezpośrednio ocenić ich dokładność i efektywność.

In [11]:
stats_path = ROOT / "data" / "european_estimator_stats.pkl"
with stats_path.open("rb") as f:
    payload = pickle.load(f)

df = pd.DataFrame(payload["stats"]).T
df

,mean,std,bias,mse,ci_coverage_95
crude,16.262920,0.087710,-0.000278,0.007685,0.945
stratified_proportional,16.259545,0.073901,-0.003653,0.005469,0.974
stratified_optimal,16.263056,0.058010,-0.000142,0.003362,0.946
antithetic,16.263128,0.067604,-0.000070,0.004566,0.987
control,16.263140,0.049019,-0.000058,0.002401,0.956


### 5.1.2. Empiryczny VRF

### 5.1.3. Wykresy

In [14]:
hist_path = PLOTS_DIR / "european_errors_hist.pkl"
box_path = PLOTS_DIR / "european_errors_box.pkl"

with hist_path.open("rb") as f:
    hist_fig = pickle.load(f)
hist_fig

In [16]:
with box_path.open("rb") as f:
    box_fig = pickle.load(f)
box_fig

### 5.1.4. Analiza i interpretacja

## 5.2. Opcja typu *up-and-out* ($C\in\{105, 120\},n=12$)

Liczba kroków dyskretyzacji wynosi $n = 12$.

porównanie crude vs te dwa stratified

### wykresy

In [13]:
hist_path = PLOTS_DIR / "barrier_errors_hist_C105.pkl"
box_path = PLOTS_DIR / "barrier_errors_box_C105.pkl"

with hist_path.open("rb") as f:
    hist_fig = pickle.load(f)
hist_fig

# 6. Dyskusja

# 7. Wnioski końcowe

# 8. Uwagi

napisać coś w stylu, że kod został tworzony razem z AI, ale był review pod kątem poprawności.

# 9. Kod źródłowy